## Initialize

In [1]:
import pandas as pd
import pulp

## Load Data

In [5]:
supply_data = pd.read_csv('supply_data.csv', index_col='node')
ship_data = pd.read_csv('ship_data.csv', index_col=['node_orig', 'node_dest'])

In [7]:
supply_data

,supply
node,
PHX,700
AUS,200
GNV,200
LAX,-200
DFW,-300
ORD,-200
ATL,-150
JFK,-250


In [8]:
ship_data

capacity  cost
node_orig node_dest                
PHX       ORD             200     6
          ATL             200     7
          DFW             200     3
          LAX             200     3
AUS       LAX             200     7
          DFW             200     2
          ATL             200     5
GNV       DFW             200     6
          ATL             200     4
          JFK             200     7
DFW       LAX             200     5
          ORD             200     4
          JFK             200     6
          ATL             200     2
ATL       JFK             200     5
          ORD             200     4
          DFW             200     2

## Build Optimization Model

In [20]:
# Sets
N = supply_data.index
O = ship_data.index.levels[0]
D = ship_data.index.levels[1]
OD = ship_data.index

In [22]:
# Parameters
supply = supply_data.loc[:,'supply']
cost = ship_data.loc[:, 'cost']
capacity = ship_data.loc[:, 'capacity']

In [26]:
# Define Lp Variable
X = pulp.LpVariable.dicts("X", ((o,d) for o,d in OD), lowBound=0, cat='Continuous')

### Model

In [28]:
# Define model
mdlLopShp = pulp.LpProblem('Cost minimizing network flow problem', pulp.LpMinimize)

/Applications/anaconda3/lib/python3.7/site-packages/pulp/pulp.py:1114: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [29]:
# Define objective
mdlLopShp += pulp.lpSum(
    [X[o,d]* cost[o,d] for o,d in OD]
)

In [31]:
# Add supply/demand constraint for every node
for n in N:
    mdlLopShp += pulp.lpSum(X.get((n,d),0) for d in D) - pulp.lpSum(X.get((o,n),0) for o in O) == supply[n]

In [33]:
# Add capacity constraint for every edge
for o,d in OD:
    mdlLopShp += X.get((o,d),0) <= capacity[o,d]

In [34]:
# Run model
mdlLopShp.solve()
pulp.LpStatus[mdlLopShp.status]

'Optimal'

### Results

In [35]:
# Minimum cost
print(pulp.value(mdlLopShp.objective))

5300.0


In [37]:
# Optimal shipment selections
for v in mdlLopShp.variables():
    print(v.name, '=', v.varValue)

X_('ATL',_'DFW') = 0.0
X_('ATL',_'JFK') = 0.0
X_('ATL',_'ORD') = 0.0
X_('AUS',_'ATL') = 0.0
X_('AUS',_'DFW') = 200.0
X_('AUS',_'LAX') = 0.0
X_('DFW',_'ATL') = 50.0
X_('DFW',_'JFK') = 50.0
X_('DFW',_'LAX') = 0.0
X_('DFW',_'ORD') = 0.0
X_('GNV',_'ATL') = 0.0
X_('GNV',_'DFW') = 0.0
X_('GNV',_'JFK') = 200.0
X_('PHX',_'ATL') = 100.0
X_('PHX',_'DFW') = 200.0
X_('PHX',_'LAX') = 200.0
X_('PHX',_'ORD') = 200.0
